<a href="https://colab.research.google.com/github/HackerJacky/TimeSeriesColab/blob/main/TS_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###時間序列第三次作業
1. 使⽤yfinance套件下載S&P500(^gspc)，台灣0050ETF(0050.tw)，比特幣(btc-usd)，黃⾦近⽉期貨價格
(gc=f)，美債三個⽉到期殖利率(^IRX)的歷史價格資料，樣本期間設定2021-01-01到2025-04-22
2. 計算每個資產的序列報酬率（使⽤對數報酬率）的平均，標準差（波動率），偏態，前1%與後1%的分位數。
3. 對上述序列報酬率進⾏平穩性檢定(ADF test)。
4. 為每個資產分別估計AR(1),AR(2),AR(3),ARMA(1,1)模型並判斷哪個模型比較好。

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
from scipy.stats import skew
from statsmodels.tsa.stattools import adfuller

In [ ]:
# 1
sp500_df  = yf.download('^GSPC',   start='2021-01-01', end='2025-04-22', threads=False)
tw50_df   = yf.download('0050.TW', start='2021-01-01', end='2025-04-22', threads=False)
btc_df    = yf.download('BTC-USD', start='2021-01-01', end='2025-04-22', threads=False)
gold_df   = yf.download('GC=F',    start='2021-01-01', end='2025-04-22', threads=False)
bonds_df  = yf.download('^IRX',    start='2021-01-01', end='2025-04-22', threads=False)

# 2️計算報酬率
sp500    = sp500_df['Close']
tw50     = tw50_df['Close']
btc      = btc_df['Close']
gold     = gold_df['Close']
bonds    = bonds_df['Close']

ret_sp    = np.log(sp500    / sp500.shift(1)).dropna()
ret_tw    = np.log(tw50     / tw50.shift(1)).dropna()
ret_btc   = np.log(btc      / btc.shift(1)).dropna()
ret_gold  = np.log(gold     / gold.shift(1)).dropna()
ret_bonds = np.log(bonds    / bonds.shift(1)).dropna()

#平均數
print("S&P500",ret_sp.mean())
print("台灣0050ETF",ret_tw.mean())
print("比特幣",ret_btc.mean())
print("黃金",ret_gold.mean())
print("債券",ret_bonds.mean())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

S&P500 Ticker
^GSPC    0.000308
dtype: float64
台灣0050ETF Ticker
0050.TW    0.000356
dtype: float64
比特幣 Ticker
BTC-USD    0.000695
dtype: float64
黃金 Ticker
GC=F    0.000519
dtype: float64
債券 Ticker
^IRX    0.003827
dtype: float64


In [ ]:
#2
##標準差(波動率)
print("S&P500",ret_sp.std())
print("台灣0050ETF",ret_tw.std())
print("比特幣",ret_btc.std())
print("黃金",ret_gold.std())
print("債券",ret_bonds.std())

S&P500 Ticker
^GSPC    0.011088
dtype: float64
台灣0050ETF Ticker
0050.TW    0.013712
dtype: float64
比特幣 Ticker
BTC-USD    0.032085
dtype: float64
黃金 Ticker
GC=F    0.009422
dtype: float64
債券 Ticker
^IRX    0.13235
dtype: float64


In [ ]:
#2
##偏態
print("S&P500",skew(ret_tw, bias=False))
print("台灣0050ETF",skew(ret_tw, bias=False))
print("比特幣",skew(ret_btc, bias=False))
print("黃金",skew(ret_gold, bias=False))
print("債券",skew(ret_bonds, bias=False))

S&P500 [-0.4635981]
台灣0050ETF [-0.4635981]
比特幣 [-0.12952661]
黃金 [-0.35367128]
債券 [0.65724945]


In [ ]:
#前後1%分位數
print(f"{'S&P500':<10} 前1%分位數: {ret_sp.quantile(0.01).iloc[0]:>8.4f} 後1%分位數: {ret_sp.quantile(0.99).iloc[0]:>8.4f}")
print(f"{'台灣0050ETF':<10} 前1%分位數: {ret_tw.quantile(0.01).iloc[0]:>8.4f} 後1%分位數: {ret_tw.quantile(0.99).iloc[0]:>8.4f}")
print(f"{'比特幣':<10} 前1%分位數: {ret_btc.quantile(0.01).iloc[0]:>8.4f} 後1%分位數: {ret_btc.quantile(0.99).iloc[0]:>8.4f}")
print(f"{'黃金':<10} 前1%分位數: {ret_gold.quantile(0.01).iloc[0]:>8.4f} 後1%分位數: {ret_gold.quantile(0.99).iloc[0]:>8.4f}")
print(f"{'債券':<10} 前1%分位數: {ret_bonds.quantile(0.01).iloc[0]:>8.4f} 後1%分位數: {ret_bonds.quantile(0.99).iloc[0]:>8.4f}")



S&P500     前1%分位數:  -0.0309 後1%分位數:   0.0254
台灣0050ETF  前1%分位數:  -0.0371 後1%分位數:   0.0352
比特幣        前1%分位數:  -0.0910 後1%分位數:   0.0944
黃金         前1%分位數:  -0.0259 後1%分位數:   0.0234
債券         前1%分位數:  -0.5108 後1%分位數:   0.5031


In [ ]:
#3 ADF test
res_SP = ts.adfuller(ret_sp,maxlag=1)
res_TW = ts.adfuller(ret_tw,maxlag=1)
res_BTC = ts.adfuller(ret_btc,maxlag=1)
res_GOLD = ts.adfuller(ret_gold,1)
res_BONDS = ts.adfuller(ret_bonds,1)

def print_adf_result(name, result):
    print(f"=== ADF Test: {name} ===")
    print(f"Test Statistic : {result[0]}")
    print(f"P-Value        : {result[1]}")
    print("Critical Values:")
    for k, v in result[4].items():
        print(f"   {k}% : {v}")
    print("-" * 30)

# 印出各資產 ADF 結果
print_adf_result("S&P500", res_SP)
print_adf_result("台灣0050", res_TW)
print_adf_result("比特幣", res_BTC)
print_adf_result("黃金", res_GOLD)
print_adf_result("債券", res_BONDS)


#因為所有產品的ADF檢定的P-value皆為0,由此可知這些資產的時間序列是平穩的

=== ADF Test: S&P500 ===
Test Statistic : -33.34460294951164
P-Value        : 0.0
Critical Values:
   1%% : -3.436436308605049
   5%% : -2.8642273405849363
   10%% : -2.568200833956216
------------------------------
=== ADF Test: 台灣0050 ===
Test Statistic : -23.6476663151138
P-Value        : 0.0
Critical Values:
   1%% : -3.4366839058058387
   5%% : -2.8643365489845647
   10%% : -2.5682589990431515
------------------------------
=== ADF Test: 比特幣 ===
Test Statistic : -41.353815448486095
P-Value        : 0.0
Critical Values:
   1%% : -3.4345219897741357
   5%% : -2.8633826834772953
   10%% : -2.5677510122114486
------------------------------
=== ADF Test: 黃金 ===
Test Statistic : -33.2758722112258
P-Value        : 0.0
Critical Values:
   1%% : -3.4364306491488814
   5%% : -2.8642248442477976
   10%% : -2.5681995044075987
------------------------------
=== ADF Test: 債券 ===
Test Statistic : -28.391676183613157
P-Value        : 0.0
Critical Values:
   1%% : -3.436441978605957
   5%% : -2.86

In [ ]:
#4
##S&P500
reg_sp_1=AutoReg(ret_sp,lags=1).fit()
reg_sp_2=AutoReg(ret_sp,lags=2).fit()
reg_sp_3=AutoReg(ret_sp,lags=3).fit()
arma_ret_sp = ARIMA(ret_sp, order=(1, 0, 1)).fit()
print("S&P500 AR(1)的AIC為",reg_sp_1.aic,"\n")
print("S&P500 AR(2)的AIC為",reg_sp_2.aic,"\n")
print("S&P500 AR(3)的AIC為",reg_sp_3.aic,"\n")
print("S&P500 ARMA(1,1)的AIC為",arma_ret_sp.aic)

#因為ARMA(1,1)的AIC值最小,所以S&P500最適合的模型為ARMA(1,1)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


S&P500 AR(1)的AIC為 -6635.494434062705 

S&P500 AR(2)的AIC為 -6626.695704420254 

S&P500 AR(3)的AIC為 -6623.414588900174 

S&P500 ARMA(1,1)的AIC為 -6639.906639926945


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
#4
##台灣0050
reg_tw_1=AutoReg(ret_tw,lags=1).fit()
reg_tw_2=AutoReg(ret_tw,lags=2).fit()
reg_tw_3=AutoReg(ret_tw,lags=3).fit()
arma_ret_tw=ARIMA(ret_tw,order=(1,0,1)).fit()
print("台灣0050 AR(1)的AIC為",reg_tw_1.aic,"\n")
print("台灣0050 AR(2)的AIC為",reg_tw_2.aic,"\n")
print("台灣0050 AR(3)的AIC為",reg_tw_3.aic,"\n")
print("台灣0050 ARMA(1,1)的AIC為",arma_ret_tw.aic)

#ARMA(1,1)的AIC值最小,由此可以判定台灣0050最適合的模型為ARMA(1,1)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


台灣0050 AR(1)的AIC為 -5942.197251195329 

台灣0050 AR(2)的AIC為 -5936.611136778509 

台灣0050 AR(3)的AIC為 -5932.358551007776 

台灣0050 ARMA(1,1)的AIC為 -5945.987875307001


In [ ]:
#4
##比特幣
reg_btc_1=AutoReg(ret_btc,lags=1).fit()
reg_btc_2=AutoReg(ret_btc,lags=2).fit()
reg_btc_3=AutoReg(ret_btc,lags=3).fit()
arma_ret_btc=ARIMA(ret_btc,order=(1,0,1)).fit()
print("比特幣 AR(1)的AIC為",reg_btc_1.aic,"\n")
print("比特幣 AR(2)的AIC為",reg_btc_2.aic,"\n")
print("比特幣 AR(3)的AIC為",reg_btc_3.aic,"\n")
print("比特幣 ARMA(1,1)的AIC為",arma_ret_btc.aic)

#AR(1)的AIC值是全部模型中最小的,所以最適合比特幣的模型為AR(1)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

比特幣 AR(1)的AIC為 -6348.54545889707 

比特幣 AR(2)的AIC為 -6343.104027611567 

比特幣 AR(3)的AIC為 -6336.843001069743 

比特幣 ARMA(1,1)的AIC為 -6344.846174052373


In [ ]:
#4
##黃金
reg_gold_1=AutoReg(ret_gold,lags=1).fit()
reg_gold_2=AutoReg(ret_gold,lags=2).fit()
reg_gold_3=AutoReg(ret_gold,lags=3).fit()
arma_ret_gold=ARIMA(ret_gold,order=(1,0,1)).fit()
print("黃金 AR(1)的AIC為",reg_gold_1.aic,"\n")
print("黃金 AR(2)的AIC為",reg_gold_2.aic,"\n")
print("黃金 AR(3)的AIC為",reg_gold_3.aic,"\n")
print("黃金 ARMA(1,1)的AIC為",arma_ret_gold.aic)
#ARMA(1,1)的AIC值是全部模型中最小的,所以最適合黃金的模型為ARMA(1,1)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


黃金 AR(1)的AIC為 -6992.304937674618 

黃金 AR(2)的AIC為 -6989.960358650014 

黃金 AR(3)的AIC為 -6981.542324508431 

黃金 ARMA(1,1)的AIC為 -6997.171226902295


In [ ]:
#4
##債券
reg_bonds_1=AutoReg(ret_bonds,lags=1).fit()
reg_bonds_2=AutoReg(ret_bonds,lags=2).fit()
reg_bonds_3=AutoReg(ret_bonds,lags=3).fit()
arma_ret_bonds=ARIMA(ret_bonds,order=(1,0,1)).fit()
print("債券 AR(1)的AIC為",reg_bonds_1.aic,"\n")
print("債券 AR(2)的AIC為",reg_bonds_2.aic,"\n")
print("債券 AR(3)的AIC為",reg_bonds_3.aic,"\n")
print("債券 ARMA(1,1)的AIC為",arma_ret_bonds.aic)

#AR(3)的AIC值是全部模型中最小的,所以最適合債券的模型為AR(3)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


債券 AR(1)的AIC為 -1298.0737358495269 

債券 AR(2)的AIC為 -1327.0418560818111 

債券 AR(3)的AIC為 -1340.9221513473483 

債券 ARMA(1,1)的AIC為 -1330.4005803246303
